# Tabular data example

In [1]:
from velour import Dataset, Model, Datum, Annotation, GroundTruth, Prediction
from velour.enums import TaskType
from velour import schemas

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
from velour.client import Client, ClientException

client = Client("http://localhost:8000")

Succesfully connected to http://localhost:8000/.


In [ ]:
dset = load_breast_cancer()
X, y, target_names = dset["data"], dset["target"], dset["target_names"]
X_train, X_test, y_train, y_test = train_test_split(X, y)

pipe = make_pipeline(StandardScaler(), LogisticRegression())

In [ ]:
X_train.shape, y_train[:4], target_names

((426, 30), array([1, 1, 0, 0]), array(['malignant', 'benign'], dtype='<U9'))

In [ ]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression())])

In [ ]:
y_train_probs = pipe.predict_proba(X_train)
y_test_probs = pipe.predict_proba(X_test)

In [ ]:
y_train_probs[:4]

array([[1.81293032e-01, 8.18706968e-01],
       [3.28318897e-03, 9.96716811e-01],
       [9.99952034e-01, 4.79656750e-05],
       [9.92904189e-01, 7.09581098e-03]])

## Velour Dataset ingestion

We now ingest the groundtruth labels into velour. For each sample, velour expects a list of `Label` objects. Each `Label` has a key and value. Allowing key/value labels and having a single row be annotated by multiple labels supports multi-label classification.

In this example there's just a single label per element and we'll set the class key to "class". The `add_groundtruth` method returns the ids of the newly created groundtruth.

Create datasets

In [ ]:
# reset (only needed if restarting each run)
client.delete_dataset("breast-cancer-train", timeout=5)
client.delete_dataset("breast-cancer-test", timeout=5)

# create or get train dataset
try:
    velour_train_dataset = Dataset.create(client, "breast-cancer-train")
except ClientException:
    velour_train_dataset = Dataset.get(client, "breast-cancer-train")

# create or get test dataset
try:
    velour_test_dataset = Dataset.create(client, "breast-cancer-test")
except ClientException:
    velour_test_dataset = Dataset.get(client, "breast-cancer-test")

Format data

In [ ]:
# format training groundtruths
training_groundtruths = [
    GroundTruth(
        datum=Datum(
            uid=f"train{i}",
        ),
        annotations=[
            Annotation(
                task_type=TaskType.CLASSIFICATION,
                labels=[schemas.Label(key="class", value=target_names[t])]
            )
        ]
    )
    for i, t in enumerate(y_train)
]

# format testing groundtruths
testing_groundtruths = [
    GroundTruth(
        datum=Datum(
            uid=f"test{i}",
        ),
        annotations=[
            Annotation(
                task_type=TaskType.CLASSIFICATION,
                labels=[schemas.Label(key="class", value=target_names[t])]
            )
        ]
    )
    for i, t in enumerate(y_test)
]

Ingest data

In [ ]:
# add the training groundtruths
for gt in training_groundtruths:
    velour_train_dataset.add_groundtruth(gt)

# add the testing groundtruths
for gt in testing_groundtruths:
    velour_test_dataset.add_groundtruth(gt)

Finalize datasets, necessary for evaluation

In [ ]:
velour_train_dataset.finalize()
velour_test_dataset.finalize()

<Response [200]>

## Model inference ingestion

Now we create a velour model and post the predictions on the two datasets. Each prediction should be a list of `ScoredLabel`, which consist of a label and a confidence score. The confidence scores over all of the classes in a key must sum to (approximately) 1.

Create model

In [ ]:
# rest (only necessary if restarting)
client.delete_model("breast-cancer-linear-model")

# create or get model
try:
    velour_model = Model.create(client, "breast-cancer-linear-model")
except ClientException:
    velour_model = Model.get(client, "breast-cancer-linear-model")

Format predictions

In [ ]:
training_predictions = [
    Prediction(
        datum=Datum(
            dataset=velour_train_dataset.name,
            uid=f"train{i}",
        ),
        annotations=[
            Annotation(
                task_type=TaskType.CLASSIFICATION,
                labels=[
                    schemas.Label(
                        key="class", 
                        value=target_names[j],
                        score=p,
                    )                        
                    for j, p in enumerate(prob)
                ]
            )
        ]
    )
    for i, prob in enumerate(y_train_probs)
]

testing_predictions = [
    Prediction(
        datum=Datum(
            dataset=velour_test_dataset.name,
            uid=f"test{i}",
        ),
        annotations=[
            Annotation(
                task_type=TaskType.CLASSIFICATION,
                labels=[
                    schemas.Label(
                        key="class",
                        value=target_names[j],
                        score=p,
                    )                        
                    for j, p in enumerate(prob)
                ]
            )
        ]
    )
    for i, prob in enumerate(y_test_probs)
]

In [ ]:
# add the train predictions
for pd in training_predictions:
    velour_model.add_prediction(pd)

# add the test predictions
for pd in testing_predictions:
    velour_model.add_prediction(pd)

finalize models, necessary for evaluation

In [ ]:
velour_model.finalize_inferences(velour_train_dataset)
velour_model.finalize_inferences(velour_test_dataset)

evaluate

In [ ]:
train_eval_job = velour_model.evaluate_classification(velour_train_dataset)
train_eval_job.wait_for_completion()

In [ ]:
train_eval_job.status

<JobStatus.DONE: 'done'>

In [ ]:
train_eval_job.metrics

{'dataset': 'breast-cancer-train',
 'model': 'breast-cancer-linear-model',
 'settings': {'task_type': 'classification'},
 'job_id': 35,
 'status': 'done',
 'metrics': [{'type': 'Accuracy',
   'parameters': {'label_key': 'class'},
   'value': 0.9859154929577465},
  {'type': 'ROCAUC',
   'parameters': {'label_key': 'class'},
   'value': 0.9971562666173447},
  {'type': 'Precision',
   'value': 0.9878787878787879,
   'label': {'key': 'class', 'value': 'malignant'}},
  {'type': 'Recall',
   'value': 0.9760479041916168,
   'label': {'key': 'class', 'value': 'malignant'}},
  {'type': 'F1',
   'value': 0.9819277108433735,
   'label': {'key': 'class', 'value': 'malignant'}},
  {'type': 'Precision',
   'value': 0.9846743295019157,
   'label': {'key': 'class', 'value': 'benign'}},
  {'type': 'Recall',
   'value': 0.9922779922779923,
   'label': {'key': 'class', 'value': 'benign'}},
  {'type': 'F1',
   'value': 0.9884615384615385,
   'label': {'key': 'class', 'value': 'benign'}}],
 'confusion_matr

In [ ]:
train_eval_job.confusion_matrices

ClientException: Not Found

In [ ]:
test_eval_job = velour_model.evaluate_classification(velour_test_dataset)
test_eval_job.wait_for_completion()

In [ ]:
test_eval_job.metrics

In [ ]:
test_eval_job.confusion_matrices

evaluation metrics in a Pandas dataframe

In [ ]:
settings_and_dfs = velour_model.get_metric_dataframes()

In [ ]:
settings = settings_and_dfs[0]["settings"]
df = settings_and_dfs[0]["df"]

In [ ]:
print(settings)

In [ ]:
df

## Sanity check scikit-learn classification report

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
y_train_preds = pipe.predict(X_train)

In [ ]:
print(classification_report(y_train, y_train_preds, digits=6, target_names=target_names))